In [120]:
#VADER Sentiment Analysis

import pandas as pd
import nltk.sentiment.sentiment_analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

reviews = pd.read_excel('Edmunds Reviews.xlsx')

reviews = reviews.drop_duplicates()

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
reviews['Review'] = reviews['Review'].str.lower()
reviews["Tokens"] = reviews["Review"].dropna().map(tokenizer.tokenize)

models = ['es','ls','rx','a8','a6','3series','5series','7series','xj','sclass']
model_set = set(models)
reviews['Intersection'] = reviews['Tokens'].map(lambda x: set(x))

reviews['Intersection'] = reviews.Intersection.map(lambda x: set.intersection(x,model_set))
reviews['Intersection'] = reviews['Intersection'].map(lambda x: list(x))

mask = reviews['Intersection'].str.len() != 0
has_models = pd.DataFrame(reviews[mask])

import re
ten_words = []
for index, row in has_models.iterrows():
    for i in row['Intersection']:
        position = row['Tokens'].index(i) 
        mod_sent = row['Tokens'][(position-5):(position+6)]
        ten_words.append(mod_sent)
		
models_sentences = []
for i in ten_words:
    models_sentences.append(" ".join(i))

has_models['Decagrams'] = pd.Series(models_sentences)

mod_sents = []
analyzer = SentimentIntensityAnalyzer()
for sentence in has_models['Decagrams']:
    vs = analyzer.polarity_scores(sentence)
    mod_sents.append(str(vs))

# When you add it to the dataframe it doesn't match up correctly for me, can someone troubleshoot this?
has_models['Model Sentiments'] = mod_sents


In [121]:
#Get compound sentiment 
def getSentiment(s):
    import ast
    sent = ast.literal_eval(s)['compound']
    return sent

has_models['Sentiment'] = has_models['Model Sentiments'].apply(getSentiment)


In [122]:
has_models[:5]

,Review,Tokens,Intersection,Decagrams,Model Sentiments,Sentiment
0,":p i like you, merc. you make me laugh! if mat...","[p, i, like, you, merc, you, make, me, laugh, ...","[rx, ls]",road what was that 2003 rx competing engineers...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,i am surprised that i did not find any discuss...,"[i, am, surprised, that, i, did, not, find, an...","[a8, 7series]",memorable gs and macbeth original ls champaign...,"{'neg': 0.0, 'neu': 0.72, 'pos': 0.28, 'compou...",0.3612
3,. . .let me see if i am following along correc...,"[let, me, see, if, i, am, following, along, co...","[a8, a6]",my main candidates are bmw 7series l and audi ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,". . .now hold on a minute. a ""current"" a6 4.2 ...","[now, hold, on, a, minute, a, current, a6, 4, ...","[a8, a6]",phaeton v8 or the audi a8 or perhaps even the a8,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
5,but has anyone noticed the front oh the new ac...,"[but, has, anyone, noticed, the, front, oh, th...",[a8],i assume the new audi a6 will come to i would,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


In [123]:
#concatenate model pairs and associated score
def getTuple(x,y):
    return x.append(y)
    
has_models.apply(lambda row: getTuple(row['Intersection'], row['Sentiment']), axis=1)

has_models['Intersection'][:5]


0            [rx, ls, 0.0]
1    [a8, 7series, 0.3612]
3            [a8, a6, 0.0]
4            [a8, a6, 0.0]
5                [a8, 0.0]
Name: Intersection, dtype: object

In [124]:
#Create edge set
edge_set = set()

for item in has_models['Intersection']:
    edge_set.add(item)

TypeError: unhashable type: 'list'